In [ ]:
!pip install selenium
!apt-get update 
!apt install chromium-chromedriver

Ign:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:2 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Hit:3 http://security.ubuntu.com/ubuntu bionic-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu bionic InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease
Hit:6 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Hit:8 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease
Hit:9 http://archive.ubuntu.com/ubuntu bionic-updates InRelease
Hit:10 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Hit:11 http://archive.ubuntu.com/ubuntu bionic-backports InRelease
Hit:12 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu bionic InRelease
Hit:13 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu bionic 

# Import Libraries

In [ ]:
import bs4 as bs
from selenium import webdriver  
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.chrome.options import Options
import sys
from selenium import webdriver
import numpy as np
import pandas as pd
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.common.by import By
from selenium.common.exceptions import TimeoutException

# Define Functions

In [ ]:
def getAdsLink(link,df):
  sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')

  chrome_options = webdriver.ChromeOptions()
  chrome_options.add_argument('--headless')
  chrome_options.add_argument('--no-sandbox')
  chrome_options.add_argument('--disable-dev-shm-usage')
  browser = webdriver.Chrome('chromedriver',options=chrome_options)
  browser.get(link)
  element = WebDriverWait(browser, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "_15tommw"))
    )
  html_source = browser.page_source

  soup = bs.BeautifulSoup(html_source, "html.parser")
  c = soup.findAll('a',{'class':'_15tommw'}, href=True)
   
  for name in c:
      #print(name['href'])
      url="https://www.airbnb.ie/"+str(name['href'])
      url_List.append(url)
  return url_List

def checkIfElementLoaded(className):
    return EC.presence_of_element_located((By.CLASS_NAME, className))

def createPageLinks(url,count,adsNum, linkList):
  #newVar="urlDublin_"+str(count)
  newOffset="&items_offset="+str(adsNum)+"&"
  newUrl=url.replace("&items_offset=0&", newOffset)
  linkList.append(newUrl)
  #print(newVar+": "+newUrl)    

def findAllElementsFromLoadedPage(soup, tag, className):
    return soup.findAll(tag, {'class':className})

def extractFeatures(url, fetList, df):
    sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
    options = webdriver.ChromeOptions()
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    options.add_argument('--disable-dev-shm-usage')
    browser = webdriver.Chrome('chromedriver', options=options)
    browser.get(url)

    element = WebDriverWait(browser, 10).until(
        checkIfElementLoaded("_fecoyn4") and # EC.presence_of_element_located((By.CLASS_NAME, "_fecoyn4"))
        checkIfElementLoaded("_1ne5r4rt") and #EC.presence_of_element_located((By.CLASS_NAME, "_1ne5r4rt"))
        checkIfElementLoaded("_142pbzop") and
        checkIfElementLoaded("_63km3vu") and
        checkIfElementLoaded("_pbq7fmm") and
        checkIfElementLoaded("_14i3z6h") and
        checkIfElementLoaded("_194e2vt2") and
        checkIfElementLoaded("_1qsawv5") and
        checkIfElementLoaded("_tyxjp1")
    )

    html_source = browser.page_source
    soup=bs.BeautifulSoup(html_source, "html.parser")

    title=findAllElementsFromLoadedPage(soup,"h1","_fecoyn4")#soup.findAll('h1', {'class':'_fecoyn4'})
    fetList=getValues(title,fetList)
    avgRating=findAllElementsFromLoadedPage(soup,"span","_1ne5r4rt")#soup.findAll('span', {'class':'_1ne5r4rt'})
    fetList=getValues(avgRating,fetList)
    numReviews=findAllElementsFromLoadedPage(soup,"span","_142pbzop")#soup.findAll('span', {'class':'_142pbzop'})
    fetList=getValues(numReviews,fetList)
    superhost=findAllElementsFromLoadedPage(soup,"span","_63km3vu")#soup.findAll('span', {'class':'_63km3vu'})
    fetList=getValues(superhost,fetList)
    location=findAllElementsFromLoadedPage(soup,"span","_pbq7fmm")#soup.findAll('span', {'class':'_pbq7fmm'})
    fetList=getValues(location,fetList)
    desc=findAllElementsFromLoadedPage(soup,"h2","_14i3z6h")
    fetList=getValues(desc,fetList)
    numofRooms=findAllElementsFromLoadedPage(soup,"ol","_194e2vt2")
    fetList=getValues(numofRooms,fetList)
    #amenities=findAllElementsFromLoadedPage(soup,"div","_1qsawv5") #multiple lines
    #getValues(amenities,fetList)
    price=findAllElementsFromLoadedPage(soup,"span","_tyxjp1")
    fetList=getValues(price,fetList)
    browser.quit()
    #df.loc[len(df)] = fetList
    df=df.append(pd.Series(fetList, index=df.columns[:len(fetList)]), ignore_index=True)
    return df
    

def getValues(tag, fetList):
  if not tag:
    var="none"
    print(var)
    fetList.append(var)
  else:
    for i in tag:
        var=i.text.strip()
        var=str(var)
        fetList.append(var)
        print(var)
  return fetList

# Define County List

In [ ]:
# Our base county list
countList=["https://www.airbnb.ie/s/County-Antrim--UK/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=april&date_picker_type=flexible_dates&query=County%20Antrim%2C%20UK&place_id=ChIJvQOTnrVMYEgR_w-BMDMWvco&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Armagh--UK/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Armagh%2C%20UK&place_id=ChIJfbQ792iTYEgRyqoxHYz9YlQ&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Carlow--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Carlow%2C%20Ireland&place_id=ChIJZbN-axJaXUgRYHYqLqfHAAQ&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Cavan--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Cavan%2C%20Ireland&place_id=ChIJLZ_1NnkSXkgRQAGA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Clare--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Clare%2C%20Ireland&place_id=ChIJGU6CyPkRW0gR8AGA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Cork/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Cork&place_id=ChIJSR2jCZLjREgRoAKA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Derry--Londonderry--United-Kingdom/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Derry%2C%20Londonderry%2C%20United%20Kingdom&place_id=ChIJTY9cWcQKYEgR4BotirI_DwM&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Donegal--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Donegal%2C%20Ireland&place_id=ChIJs7A6kGSWX0gRACCY8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Down--United-Kingdom/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Down%2C%20United%20Kingdom&place_id=ChIJh3HgpQQJYUgRc9Gxq_7NAfI&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Dublin/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&query=County%20Dublin&place_id=ChIJv2RI7foRZ0gRwAKA8azHABg&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Fermanagh--UK/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Fermanagh%2C%20UK&place_id=ChIJyxsZOB9GXkgREBstirI_DwM&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Galway/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Galway&place_id=ChIJrx2gLKaVW0gR0AKA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Kerry--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Kerry%2C%20Ireland&place_id=ChIJRy3pzLhPRUgRAB6Y8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Kildare--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Kildare%2C%20Ireland&place_id=ChIJR3HKcbV_XUgR8AKA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Kilkenny--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Kilkenny%2C%20Ireland&place_id=ChIJ4XFVRPcxXUgRAAOA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/Laois--County-Laois--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=Laois%2C%20County%20Laois%2C%20Ireland&place_id=ChIJG5glBMsNXUgRv42wYAzqMxw&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Leitrim--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Leitrim%2C%20Ireland&place_id=ChIJdeoGgpOKXkgRcHUqLqfHAAM&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Limerick/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Limerick&place_id=ChIJOwx_mjJPW0gRcAGA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Longford--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Longford%2C%20Ireland&place_id=ChIJXyOBkkEuXEgR3vrf5usRzs0&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Louth--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Louth%2C%20Ireland&place_id=ChIJa854tbI0Z0gRYOS7M8JgTks&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Mayo--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Mayo%2C%20Ireland&place_id=ChIJJ1YLjJJoWUgRoAGA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Meath--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Meath%2C%20Ireland&place_id=ChIJv1cLaulbZ0gRsAGA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Monaghan--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Monaghan%2C%20Ireland&place_id=ChIJ--AL-UKfYEgRT7glEhngANY&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Offaly--Ireland/homes?tab_id=home_tab&refinement_paths[]=%2Fhomes&flexible_trip_dates[]=april&flexible_trip_dates[]=december&flexible_trip_dates[]=february&flexible_trip_dates[]=january&flexible_trip_dates[]=march&flexible_trip_dates[]=may&date_picker_type=flexible_dates&flexible_trip_lengths[]=one_week&source=structured_search_input_header&search_type=search_query&query=County%20Offaly&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Roscommon--Ireland/homes?tab_id=home_tab&refinement_paths[]=%2Fhomes&flexible_trip_dates[]=april&flexible_trip_dates[]=december&flexible_trip_dates[]=february&flexible_trip_dates[]=january&flexible_trip_dates[]=march&flexible_trip_dates[]=may&date_picker_type=flexible_dates&flexible_trip_lengths[]=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=Roscommon%2C%20County%20Roscommon%2C%20Ireland&place_id=ChIJf3lhIcgYXEgRgHcqLqfHAAQ&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Sligo--Ireland/homes?tab_id=home_tab&refinement_paths[]=%2Fhomes&flexible_trip_dates[]=april&flexible_trip_dates[]=december&flexible_trip_dates[]=february&flexible_trip_dates[]=january&flexible_trip_dates[]=march&flexible_trip_dates[]=may&date_picker_type=flexible_dates&flexible_trip_lengths[]=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=Sligo%2C%20County%20Sligo%2C%20Ireland&place_id=ChIJmaHiaIzGXkgRkHcqLqfHAAQ&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Tipperary/homes?tab_id=home_tab&refinement_paths[]=%2Fhomes&flexible_trip_dates[]=april&flexible_trip_dates[]=december&flexible_trip_dates[]=february&flexible_trip_dates[]=january&flexible_trip_dates[]=march&flexible_trip_dates[]=may&date_picker_type=flexible_dates&flexible_trip_lengths[]=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=Tipperary%2C%20County%20Tipperary&place_id=ChIJlTpMqmiyXEgRYHUxl6nHAAo&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/Tyrone--Northern-Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=Tyrone%2C%20Northern%20Ireland&place_id=ChIJdWfzpUt-YEgRdmB-gxL1mis&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Waterford--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Waterford%2C%20Ireland&place_id=ChIJS14ZGTgiQ0gRIAKA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Westmeath--Ireland/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Westmeath%2C%20Ireland&place_id=ChIJt7ZDotLGXUgRMAKA8azHABg&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Wexford--Ireland/homes?tab_id=home_tab&refinement_paths[]=%2Fhomes&flexible_trip_dates[]=april&flexible_trip_dates[]=december&flexible_trip_dates[]=february&flexible_trip_dates[]=january&flexible_trip_dates[]=march&flexible_trip_dates[]=may&date_picker_type=flexible_dates&flexible_trip_lengths[]=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=Wexford%2C%20County%20Wexford%2C%20Ireland&place_id=ChIJ8QOaQbwBaEgR0HcqLqfHAAQ&pagination_search=true&items_offset=0&section_offset=2",
          "https://www.airbnb.ie/s/County-Wicklow--Ireland/homes?tab_id=home_tab&refinement_paths[]=%2Fhomes&flexible_trip_dates[]=april&flexible_trip_dates[]=december&flexible_trip_dates[]=february&flexible_trip_dates[]=january&flexible_trip_dates[]=march&flexible_trip_dates[]=may&date_picker_type=flexible_dates&flexible_trip_lengths[]=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=Wicklow%2C%20County%20Wicklow%2C%20Ireland&place_id=ChIJu_laKAKUZ0gR4HcqLqfHAAQ&pagination_search=true&items_offset=0&section_offset=2"]

In [ ]:
len(countList)

32

# Create Page Links for all counties

In [ ]:
count=1
adsNum=0
linkList=[]

for url in countList:  
  for i in range(1, 16):
    createPageLinks(url,count,adsNum,linkList)
    adsNum=adsNum+20
    count=count+1

In [ ]:
len(linkList)

480

In [ ]:
linkList

['https://www.airbnb.ie/s/County-Antrim--UK/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=april&date_picker_type=flexible_dates&query=County%20Antrim%2C%20UK&place_id=ChIJvQOTnrVMYEgR_w-BMDMWvco&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&pagination_search=true&items_offset=0&section_offset=2',
 'https://www.airbnb.ie/s/County-Antrim--UK/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&flexible_trip_dates%5B%5D=april&date_picker_type=flexible_dates&query=County%20Antrim%2C%20UK&place_id=ChIJvQOTnrVMYEgR_w-BMDMWvco&flexible_trip_lengths%5B%5D=one_week&source=str

In [ ]:

url=countList[10]

count=1
adsNum=0
linkList=[]

In [ ]:
for i in range(1, 16):
    createPageLinks(url,count,adsNum,linkList)
    adsNum=adsNum+20
    count=count+1

In [ ]:
linkList

['https://www.airbnb.ie/s/County-Fermanagh--UK/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&query=County%20Fermanagh%2C%20UK&place_id=ChIJyxsZOB9GXkgREBstirI_DwM&pagination_search=true&items_offset=0&section_offset=2',
 'https://www.airbnb.ie/s/County-Fermanagh--UK/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click

In [ ]:
len(linkList)

15

## Create Ad Links for All Pages

In [ ]:
url_List=[]

for link in linkList:
  getAdsLink(link,url_List)

In [ ]:
url_List

['https://www.airbnb.ie//rooms/20991046?adults=1&children=0&infants=0&check_in=2021-12-05&check_out=2021-12-12&previous_page_section_name=1000&federated_search_id=dfc7e6a3-0789-4e6d-b2da-ea8c5cc8b336',
 'https://www.airbnb.ie//rooms/39739322?adults=1&children=0&infants=0&check_in=2022-01-09&check_out=2022-01-16&previous_page_section_name=1000&federated_search_id=dfc7e6a3-0789-4e6d-b2da-ea8c5cc8b336',
 'https://www.airbnb.ie//rooms/25120535?adults=1&children=0&infants=0&check_in=2022-05-20&check_out=2022-05-27&previous_page_section_name=1000&federated_search_id=dfc7e6a3-0789-4e6d-b2da-ea8c5cc8b336',
 'https://www.airbnb.ie//rooms/3437196?adults=1&children=0&infants=0&check_in=2022-05-21&check_out=2022-05-28&previous_page_section_name=1000&federated_search_id=dfc7e6a3-0789-4e6d-b2da-ea8c5cc8b336',
 'https://www.airbnb.ie//rooms/14978129?adults=1&children=0&infants=0&check_in=2022-02-07&check_out=2022-02-14&previous_page_section_name=1000&federated_search_id=dfc7e6a3-0789-4e6d-b2da-ea8c5c

### Specific Solutions for problematic links

In [ ]:
# for 9
link9Dublin="https://www.airbnb.ie/s/County-Dublin/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&query=County%20Dublin&place_id=ChIJv2RI7foRZ0gRwAKA8azHABg&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&pagination_search=true&items_offset=0&section_offset=2"

count=1
adsNum=0
linkList=[]

for i in range(1, 16):
  createPageLinks(link9Dublin,count,adsNum,linkList)
  adsNum=adsNum+20
  count=count+1

In [ ]:
linkList

['https://www.airbnb.ie/s/County-Dublin/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&query=County%20Dublin&place_id=ChIJv2RI7foRZ0gRwAKA8azHABg&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_header&search_type=autocomplete_click&pagination_search=true&items_offset=0&section_offset=2',
 'https://www.airbnb.ie/s/County-Dublin/homes?tab_id=home_tab&refinement_paths%5B%5D=%2Fhomes&flexible_trip_dates%5B%5D=december&flexible_trip_dates%5B%5D=january&flexible_trip_dates%5B%5D=february&flexible_trip_dates%5B%5D=march&flexible_trip_dates%5B%5D=april&flexible_trip_dates%5B%5D=may&date_picker_type=flexible_dates&query=County%20Dublin&place_id=ChIJv2RI7foRZ0gRwAKA8azHABg&flexible_trip_lengths%5B%5D=one_week&source=structured_search_input_hea

In [ ]:
print(len(linkList))

15


## Creating Data Frame

In [ ]:
df=pd.DataFrame(columns=['TITLE','AVERAGE RATING','NUMBER OF REVIEWS','SUPERHOST','LOCATION','DESCRIPTION','NUMBER OF ROOMS','PRICE PER NIGHT','ADDITIONAL COLUMN','ADDITIONAL COLUMN2','ADDITIONAL COLUMN3'])
df

,TITLE,AVERAGE RATING,NUMBER OF REVIEWS,SUPERHOST,LOCATION,DESCRIPTION,NUMBER OF ROOMS,PRICE PER NIGHT,ADDITIONAL COLUMN,ADDITIONAL COLUMN2,ADDITIONAL COLUMN3


## Get Features

In [ ]:
for url in url_List:
  fetList=[]
  df=extractFeatures(url,fetList,df)

Riverside setting 5 mins. walk to our island town
4.82
(93 reviews)
Superhost
Fermanagh, Northern Ireland, United Kingdom
Entire rental unit hosted by Sean + Susan
4 guests · 2 bedrooms · 3 beds · 1 bathroom
€49
Traditional Irish Thatched  Cottage
4.92
(280 reviews)
Superhost
Fermanagh, Northern Ireland, United Kingdom
Entire cottage hosted by Martel
4 guests · 1 bedroom · 4 beds · 1 bathroom
€47
Beautiful Apartment for two
4.86
(207 reviews)
Superhost
Lisnaskea, Northern Ireland, United Kingdom
Entire rental unit hosted by Robyn
2 guests · 1 bedroom · 1 bed · 1 bathroom
€50
Darling Street Apartments
4.50
(37 reviews)
none
Fermanagh, Northern Ireland, United Kingdom
Entire apartment hosted by Alison
4 guests · 2 bedrooms · 4 beds · 1 bathroom
€60
Riverview House
4.68
(224 reviews)
none
Enniskillen, Northern Ireland, United Kingdom
Entire rental unit hosted by Brian
4 guests · 2 bedrooms · 3 beds · 1 bathroom
€53
24 Castle Hume Court Enniskillen Holiday Apartment
5.0
(7 reviews)
none
Fe

In [ ]:
df.to_csv("County10.csv")